In [1]:
import numpy as np
import pandas as pd
import scipy.linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Ellipse, Rectangle
import matplotlib.transforms as transforms
import matplotlib as mpl
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 8}

plt.rc('font', **font)
width  = 3.402
height = width / 1.2

import sys
print(sys.version)
sys.path.append('../')
import time

from models.powertrain.bounded_powertrain import Bounded_powertrain
from models.kinematic.ideal_diff_drive import Ideal_diff_drive
from models.learning.blr_slip import SlipBayesianLinearRegression, FullBodySlipBayesianLinearRegression
from models.learning.blr_slip_acceleration import SlipAccelerationBayesianLinearRegression, FullBodySlipAccelerationBayesianLinearRegression
from models.learning.blr_acceleration import AccelerationBayesianLinearRegression, FullBodyAccelerationBayesianLinearRegression
from models.learning.gp_slip import *
from models.kinematic.ICR_based import *
from models.kinematic.Perturbed_unicycle import *
from models.kinematic.enhanced_kinematic import *


from util.transform_algebra import *
from util.util_func import *

from sklearn import linear_model

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [2]:
# import all slip datasets

train_datasets = []
validate_datasets = []
full_datasets = []

dataset_paths = []
dataset_labels = []
dataset_paths.append('../data/ral2023_dataset/husky/boreal_snow/acceleration_dataset.pkl')
dataset_labels.append(r'Hu_snow')
dataset_paths.append('../data/ral2023_dataset/husky/grand_salon_tile_inflated/acceleration_dataset.pkl')
dataset_labels.append(r'Hu_tile')
# dataset_paths.append('../data/ral2023_dataset/husky/grand_salon_left-deflated/acceleration_dataset.pkl')
# dataset_labels.append('H_tile_ld')
dataset_paths.append('../data/ral2023_dataset/marmotte/boreal_snow/acceleration_dataset.pkl')
dataset_labels.append(r'HD2_snow')
        # dataset_paths.append('../data/ral2023_dataset/marmotte/ga_hard_snow_a/acceleration_dataset.pkl')
        # dataset_labels.append('M_hard_snow_a')
dataset_paths.append('../data/ral2023_dataset/marmotte/grand_salon_tile_b/acceleration_dataset.pkl')
dataset_labels.append(r'HD2_tile')
# dataset_paths.append('../data/ral2023_dataset/marmotte/ga_hard_snow_b/acceleration_dataset.pkl')
# dataset_labels.append('M_hard_snow')
# dataset_paths.append('../data/ral2023_dataset/warthog_tracks/boreal_mud/acceleration_dataset.pkl')
# dataset_labels.append(r'Wt_mud')
# dataset_paths.append('../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/acceleration_dataset.pkl')
# dataset_labels.append('Wt_crust')
dataset_paths.append('../data/ral2023_dataset/warthog_wheels/gravel_1/acceleration_dataset.pkl')
dataset_labels.append(r'Ww_gravel')
dataset_paths.append('../data/ral2023_dataset/warthog_wheels/ice/acceleration_dataset.pkl')
dataset_labels.append(r'Ww_ice')

for dataset_path in dataset_paths:
    full_dataset = pd.read_pickle(dataset_path)
    full_dataset_length = len(full_dataset)
    train_dataset = full_dataset[:int(full_dataset_length/2)]
    train_datasets.append(train_dataset)
    validate_dataset = full_dataset[int(full_dataset_length/2):]
    validate_datasets.append(validate_dataset)
    full_datasets.append(full_dataset)
    print(dataset_path)


# print(train_dataset)

../data/ral2023_dataset/husky/boreal_snow/acceleration_dataset.pkl
../data/ral2023_dataset/husky/grand_salon_tile_inflated/acceleration_dataset.pkl
../data/ral2023_dataset/marmotte/boreal_snow/acceleration_dataset.pkl
../data/ral2023_dataset/marmotte/grand_salon_tile_b/acceleration_dataset.pkl
../data/ral2023_dataset/warthog_wheels/gravel_1/acceleration_dataset.pkl
../data/ral2023_dataset/warthog_wheels/ice/acceleration_dataset.pkl


In [3]:
# import powertrain vels

cmd_left_str_list = []
cmd_right_str_list = []
for i in range(0, 40):
    str_cmd_left_i = 'cmd_left_' + str(i)
    str_cmd_right_i = 'cmd_right_' + str(i)
    cmd_left_str_list.append(str_cmd_left_i)
    cmd_right_str_list.append(str_cmd_right_i)
# cmd_left_vels_array = validate_dataset[cmd_left_str_list].to_numpy()
# cmd_right_vels_array = validate_dataset[cmd_right_str_list].to_numpy()

encoder_left_str_list = []
encoder_right_str_list = []
for i in range(0, 40):
    str_encoder_left_i = 'left_wheel_vel_' + str(i)
    str_encoder_right_i = 'right_wheel_vel_' + str(i)
    encoder_left_str_list.append(str_encoder_left_i)
    encoder_right_str_list.append(str_encoder_right_i)
# encoder_left_vels_array = validate_dataset[encoder_left_str_list].to_numpy()
# encoder_right_vels_array = validate_dataset[encoder_right_str_list].to_numpy()

transitory_left_str_list = []
transitory_right_str_list = []
for i in range(0, 40):
    str_transitory_left_i = 'transitory_vel_left_' + str(i)
    str_transitory_right_i = 'transitory_vel_right_' + str(i)
    transitory_left_str_list.append(str_transitory_left_i)
    transitory_right_str_list.append(str_transitory_right_i)
# transitory_left_vels_array = validate_dataset[transitory_left_str_list].to_numpy()
# transitory_right_vels_array = validate_dataset[transitory_right_str_list].to_numpy()



In [4]:
# extract cmd_body_vel arrays (input arrays)

idd_body_vel_x_str_list = []
idd_body_vel_y_str_list = []
idd_body_vel_yaw_str_list = []
for i in range(0, 40):
    str_idd_vel_x_i = 'idd_vel_x_' + str(i)
    str_idd_vel_y_i = 'idd_vel_y_' + str(i)
    str_idd_vel_yaw_i = 'idd_vel_yaw_' + str(i)
    idd_body_vel_x_str_list.append(str_idd_vel_x_i)
    idd_body_vel_y_str_list.append(str_idd_vel_y_i)
    idd_body_vel_yaw_str_list.append(str_idd_vel_yaw_i)
# idd_body_vel_x_array = validate_dataset[idd_body_vel_x_str_list].to_numpy()
# idd_body_vel_y_array = validate_dataset[idd_body_vel_y_str_list].to_numpy()
# idd_body_vel_yaw_array = validate_dataset[idd_body_vel_yaw_str_list].to_numpy()

icp_x_str_list = []
icp_y_str_list = []
icp_z_str_list = []
icp_roll_str_list = []
icp_pitch_str_list = []
icp_yaw_str_list = []
for i in range(0, 40):
    str_icp_x_i = 'icp_interpolated_x_' + str(i)
    str_icp_y_i = 'icp_interpolated_y_' + str(i)
    str_icp_yaw_i = 'icp_interpolated_yaw_' + str(i)
    icp_x_str_list.append(str_icp_x_i)
    icp_y_str_list.append(str_icp_y_i)
    icp_yaw_str_list.append(str_icp_yaw_i)
# icp_x_array = validate_dataset[icp_x_str_list].to_numpy()
# icp_y_array = validate_dataset[icp_y_str_list].to_numpy()
# icp_yaw_array = validate_dataset[icp_yaw_str_list].to_numpy()

# extract icp vels

str_icp_vel_x_list = []
str_icp_vel_y_list = []
str_icp_vel_yaw_list = []
for i in range(0, 40):
    str_icp_vel_x_i = 'icp_vel_x_' + str(i)
    str_icp_vel_y_i = 'icp_vel_y_' + str(i)
    str_icp_vel_yaw_i = 'icp_vel_yaw_' + str(i)
    str_icp_vel_x_list.append(str_icp_vel_x_i)
    str_icp_vel_y_list.append(str_icp_vel_y_i)
    str_icp_vel_yaw_list.append(str_icp_vel_yaw_i)
icp_vel_x_array = validate_dataset[str_icp_vel_x_list].to_numpy()
icp_vel_y_array = validate_dataset[str_icp_vel_y_list].to_numpy()
icp_vel_yaw_array = validate_dataset[str_icp_vel_yaw_list].to_numpy()

# extract body_vel_distruptions arrays (output arrays)

str_body_vel_disturption_x_list = []
str_body_vel_disturption_y_list = []
str_body_vel_disturption_yaw_list = []
for i in range(0, 40):
    str_body_vel_disturption_x_i = 'body_vel_disturption_x_' + str(i)
    str_body_vel_disturption_y_i = 'body_vel_disturption_y_' + str(i)
    str_body_vel_disturption_yaw_i = 'body_vel_disturption_yaw_' + str(i)
    str_body_vel_disturption_x_list.append(str_body_vel_disturption_x_i)
    str_body_vel_disturption_y_list.append(str_body_vel_disturption_y_i)
    str_body_vel_disturption_yaw_list.append(str_body_vel_disturption_yaw_i)

# body_vel_disturption_x_array = validate_dataset[str_body_vel_disturption_x_list].to_numpy()
# body_vel_disturption_y_array = validate_dataset[str_body_vel_disturption_y_list].to_numpy()
# body_vel_disturption_yaw_array = validate_dataset[str_body_vel_disturption_yaw_list].to_numpy()

In [6]:
def compute_stats_dataset(dataset):

    icp_x_array = dataset[icp_x_str_list].to_numpy()
    icp_y_array = dataset[icp_y_str_list].to_numpy()
    icp_yaw_array = dataset[icp_yaw_str_list].to_numpy()

    traveled_distance = 0

    n_windows = int(len(dataset)/3)

    dataset_time = 6 * n_windows / 60**2 # hours

    for i in range(0, n_windows):
        for j in range(1, 40):
            current_icp_state = np.array([icp_x_array[i, j], icp_y_array[i, j], icp_yaw_array[i, j]])
            previous_icp_state = np.array([icp_x_array[i, j-1], icp_y_array[i, j-1], icp_yaw_array[i, j-1]])
            traveled_distance += comp_disp(current_icp_state, previous_icp_state)

    return traveled_distance, dataset_time

total_traveled_distance = 0
total_elapsed_time = 0
for dataset in full_datasets:
    traveled_distance, dataset_time = compute_stats_dataset(dataset)
    total_traveled_distance += traveled_distance
    total_elapsed_time += dataset_time

print('total traveled distance [m] : ' + str(total_traveled_distance))
print('total elapsed time [hours] : ' + str(total_elapsed_time))

total traveled distance [m] : 2410.8077143271
total elapsed time [hours] : 1.7799999999999998


In [4]:
# compute area loss

def rectangle_area(width, height):
    return width * height

def diamond_area(width, height):
    return width * height / 2

def relative_area_loss(init_area, result_area):
    return 100 * (init_area - result_area) / init_area

maximum_linear_vel_positive = 5
maximum_linear_vel_negative = -5
maximum_angular_vel_positive = 4
maximum_angular_vel_negative = -4
gravel_maximum_linear_vel_positive = 4
gravel_maximum_linear_vel_negative = -4
gravel_maximum_angular_vel_positive = 2.5
gravel_maximum_angular_vel_negative = -2.5
snow_maximum_linear_vel_positive = 2.5
snow_maximum_linear_vel_negative = -2.5
snow_maximum_angular_vel_positive = 1
snow_maximum_angular_vel_negative = -1
human_maximum_linear_vel_positive = 5
human_maximum_linear_vel_negative = 0
human_maximum_angular_vel_positive = 2
human_maximum_angular_vel_negative = -2

naive_area = rectangle_area((maximum_angular_vel_positive - maximum_angular_vel_negative),
                            (maximum_linear_vel_positive - maximum_linear_vel_negative))
input_space_area = diamond_area((maximum_angular_vel_positive - maximum_angular_vel_negative),
                            (maximum_linear_vel_positive - maximum_linear_vel_negative))
input_space_area_loss = relative_area_loss(naive_area, input_space_area)

gravel_area = diamond_area((gravel_maximum_angular_vel_positive - gravel_maximum_angular_vel_negative),
                           (gravel_maximum_linear_vel_positive - gravel_maximum_linear_vel_negative))
gravel_area_loss = relative_area_loss(naive_area, gravel_area)

snow_area = diamond_area((snow_maximum_angular_vel_positive - snow_maximum_angular_vel_negative),
                           (snow_maximum_linear_vel_positive - snow_maximum_linear_vel_negative))
snow_area_loss = relative_area_loss(naive_area, snow_area)

print('input-space area loss : ' + str(input_space_area_loss) + '%')
print('gravel area loss : ' + str(gravel_area_loss) + '%')
print('snow area loss : ' + str(snow_area_loss) + '%')


input-space area loss : 50.0%
gravel area loss : 75.0%
snow area loss : 93.75%
